In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("../output/loss.csv")
df.head()

,step,epoch,mesa,base
0,0,0,12.497458,10.910637
1,1,0,8.819035,7.821729
2,2,0,10.557375,7.866491
3,3,0,8.533783,8.117002
4,4,0,12.024865,10.225608


In [6]:
loss1, loss2 = 'mesa', 'base'
# Create scatter plot
fig = go.Figure()
# Add scatter plot for Column1 vs Column2
fig.add_trace(go.Scatter(
    x=df[loss1], 
    y=df[loss2], 
    mode='markers', 
    marker=dict(color='blue'), 
    name=f'{loss1} vs {loss2}'
))
# Update layout for fonts
fig.update_layout(
    title=f'{loss1} vs {loss2}',
    xaxis_title=loss1,
    yaxis_title=loss2,
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="black"
    )
)

In [8]:
# Save the plot as a PDF
fig.write_image("../output/loss.pdf", format="pdf")